<a href="https://colab.research.google.com/github/jainmadhur07/KnowYourAudience-UserAnalysis/blob/main/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai supabase

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.9/169.9 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.15
    Uninstalling aiohttp-3.11.15:
      Successfully uninstalled aiohttp-3.11.15


In [ ]:
import google.generativeai as genai
from supabase import create_client, Client

In [ ]:
SUPABASE_URL = ""
SUPABASE_KEY = ""
GEMINI_API_KEY = ""

In [ ]:
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")

In [ ]:
# Fetch user profiles and chat history
response = supabase.table("user_profiles").select("*").execute()
user_profiles = response.data

chat_response = supabase.table("chat_history").select("*").execute()
chat_history = chat_response.data

print(f"Found {len(user_profiles)} user profiles")
print(f"Found {len(chat_history)} chat history records")

Found 5 user profiles
Found 5 chat history records


In [ ]:
# Process each user
for profile in user_profiles:
    user_id = profile['user_id']

    # Get user's chat history
    user_chat_logs = []
    for chat_record in chat_history:
        if chat_record['user_id'] == user_id and 'logs' in chat_record:
            user_chat_logs.extend(chat_record['logs'])
            break

    if not user_chat_logs:
        continue

    # Prepare data for classification
    try:
        user_data = {
            "user_id": user_id,
            "total_questions": profile['total_questions'],
            "active_topics": profile['active_topics'],
            "most_common_question_type": profile['most_common_question_type'],
            "avg_time_per_question": profile['avg_time_per_question'],
            "avg_time_per_session": profile.get('avg_time_per_session', profile.get('avg_questions_per_session', 0)),
            "recent_questions": [
                {
                    "prompt": log['prompt'],
                    "topic": log['topic'],
                    "question_type": log['question_type'],
                    "time": log['time'],
                    "is_followup": log['is_followup']
                }
                for log in user_chat_logs[-5:]
            ]
        }

        # Create classification prompt
        recent_questions_str = ""
        for q in user_data['recent_questions']:
            recent_questions_str += f"- {q['prompt']} (Topic: {q['topic']}, Type: {q['question_type']}, Time: {q['time']}s)\n"

        prompt = f"""Classify this user as BEGINNER, INTERMEDIATE, or ADVANCED based on their learning patterns:

User Data:
- Total Questions: {user_data['total_questions']}
- Active Topics: {user_data['active_topics']}
- Most Common Question Type: {user_data['most_common_question_type']}
- Average Time per Question: {user_data['avg_time_per_question']} seconds
- Average Time per Session: {user_data['avg_time_per_session']} seconds

Recent Questions and Topics:
{recent_questions_str}

Classification Criteria:
Note: Advanced users typically ask fewer(<=2 questions) but more complex, targeted questions
Respond with only: BEGINNER, INTERMEDIATE, or ADVANCED"""

        # Get classification from Gemini
        result = model.generate_content(prompt)
        classification = result.text.strip().upper()

        # Ensure valid classification
        if classification not in ['BEGINNER', 'INTERMEDIATE', 'ADVANCED']:
            classification = 'BEGINNER'

        print(f"{user_id}: {classification}")

    except Exception as e:
        print(f"Error processing {user_id}: {e}")
        continue

user_1: BEGINNER
user_2: BEGINNER
user_3: ADVANCED
user_4: BEGINNER
user_5: ADVANCED
